# Testing tensor contraction on a batch of matrices.
<p>Program tests the parallel execution of a large number of matrix-matrix multiplications of tall-and-skinny matrices of the form</p>

$\Psi_{ik} \times \Psi^*_{ki}$

<p>of dimension ($2^k, 2^{n-k}$) where $2 <= k <= n/2$. The goal is test which is the maximum $n$ for a given architecture that can be computed.
The matrices are double complex. </p>

The current operating dimension is $n = 12$ but the goal would be $23$.


In [1]:
import numpy as np
import sys
import scipy.special
import time

**Fix parameters $n$ and $k$, compute the matrix size and the number of matrix products (matrix pairs) that has to be computed**

In [21]:
n = 27
k = 13
num_rows = pow(2,k)
num_cols = pow(2,n-k)
num_matrices = scipy.special.comb(n, k, exact=True)
print('Number of matrices of size (' + str(num_rows) + ' x ' + str(num_cols) + ') is ' + str(num_matrices))

Number of matrices of size (8192 x 16384) is 20058300


**Generate a complex matrix of dimension $2^k \times 2^{n-k}$**

In [22]:
A = np.empty([num_rows, num_cols], dtype=np.complex128)
A.real = np.array(np.random.rand(num_rows, num_cols), dtype=np.float64)-0.5
A.imag = np.array(np.random.rand(num_rows, num_cols), dtype=np.float64)-0.5

**Compute matrix-matrix product of all matrix pairs**

In [4]:
tic = time.perf_counter()
for iter in np.arange(num_matrices):
    A.dot(A.transpose())
toc = time.perf_counter()
tot = toc - tic
print('Total time: ' + str(tot) + ' sec')

Total time: 9.245614607003517 sec


In [5]:
print(sys.executable)
!which python
!{sys.executable} -m pip install --upgrade pip

/usr/bin/python3.6
which: no python in (/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin)
Defaulting to user installation because normal site-packages is not writeable


In [6]:
!{sys.executable} -m pip install --user "dask[complete]"

In [27]:
sys.path.append('/home/jupiter/.local/bin')

import dask.array as da
dask_A = da.asanyarray(A)
dask_A = da.rechunk(dask_A)
dask_A

dask.array<rechunk-merge, shape=(8192, 16384), dtype=complex128, chunksize=(2048, 4096), chunktype=numpy.ndarray>

In [21]:
from dask.distributed import Client
client = Client()

tic = time.perf_counter()

for iter in np.arange(num_matrices):
    future = client.submit(A.dot(A.transpose()))
toc = time.perf_counter()
tot = toc - tic
print('Total time: ' + str(tot) + ' sec')

ModuleNotFoundError: No module named 'dask'